In [168]:
import pandas as pd
import os
import json
import numpy as np
import re

In [169]:
## reading the Survey File
#surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\Test.csv",sep="," , skiprows=1)
surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\AmenityClubSurvey FINAL v2 Season Cleaned_deletedFirstTwoColumn.csv",sep="," , skiprows=1, keep_default_na=False)
surveyData.head()

## Reading the Config File and Extracting the Information from there
config_file = open("C:\Projects\MSS\Loading _AmenityData_from_Survey\Python_Codes\ClubCorp\GolfServices\GolfServicesColMapping.json")
golf_config = json.load(config_file)

##
columnName = ''

In [170]:
def determineBooleanValues (dataFrame):
    val = False
    if pd.notnull(dataFrame[columnName]) :
        if dataFrame[columnName] == columnName:
            val = True
    return val

In [171]:
def prepareHoursOfOperationJson(season, seasonStartDate, seasonStartMonth, seasonEndDate, seasonEndMonth, openDays, startTime, endTime):
    
    hoursOfOperation = {}
    hoursOfOperationValue = {}

    ## adding season value
    if season and not pd.isnull(season):
        seasonValue = {}
        seasonValue["value"] = season
        hoursOfOperationValue["Season"] = [seasonValue]

    ## Adding season start
    seasonStart = {}
    seasonStartValue = {}
    
    if seasonStartDate and not pd.isnull(seasonStartDate):
        date ={}
        date["value"] = seasonStartDate
        seasonStartValue["Date"] = [date]
   
    if seasonStartMonth and not pd.isnull(seasonStartMonth):
        month ={}
        month["value"] = seasonStartMonth
        seasonStartValue["Month"] = [month]
    
    
    if seasonStartValue :
        seasonStart["value"] = seasonStartValue
        hoursOfOperationValue["SeasonStart"] = [seasonStart]


    ## Adding season end
    seasonEnd = {}
    seasonEndValue = {}
    
    if seasonEndDate and not pd.isnull(seasonEndDate): 
        date ={}
        date["value"] = seasonEndDate
        seasonEndValue["Date"] = [date]
    
    if seasonEndMonth and not pd.isnull(seasonEndMonth):
        month ={}
        month["value"] = seasonEndMonth
        seasonEndValue["Month"] = [month]
        
    if seasonEndValue :
        seasonEnd["value"] = seasonEndValue
        hoursOfOperationValue["SeasonEnd"] = [seasonEnd]
    
    ## Adding Day and Timing
    dayAndTiming = {}
    dayAndTimingValue = {}
    
    for day in openDays :
        dayValue = {}
        dayValue["value"] = True
        dayAndTimingValue[day] = [dayValue]
        
    
    
    timing = {}
    timingValue = {}
    
    if startTime and not pd.isnull(startTime): 
        startTimeDict ={}
        startTimeDict["value"] = startTime
        timingValue["StartTime"] =  [startTimeDict]
        
    if endTime and not pd.isnull(endTime):
        endTimeDict ={}
        endTimeDict["value"] = endTime
        timingValue["EndTime"] =  [endTimeDict]
    
    ## Adding day part as "All Day"
    dayPart ={}
    dayPart["value"] = "All Day"
    timingValue["DayPart"] =  [dayPart]
    
    
    if timingValue :
        timing["value"] = timingValue
        dayAndTimingValue["Timing"] = [timing]
    
    if dayAndTimingValue : 
        dayAndTiming["value"] = dayAndTimingValue
        hoursOfOperationValue["DayAndTiming"] = [dayAndTiming]
    
    if hoursOfOperationValue :
        hoursOfOperation["value"] = hoursOfOperationValue
    
    return hoursOfOperation

In [172]:
def createHoursOfOperationForEachRow(hoursOfOperationInfo, surveyData, isErrorSavingRequired):
    
    errorData = []
    allHoursOfOperation = []
    allDays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    allMonths = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    
    months = {"Jan" : "January","Feb" : "February","Mar" : "March", "Apr" : "April", "May" : "May", "Jun" : "June", "Jul" : "July", "Aug" : "August", "Sep" : "September", "Oct" : "October", "Nov" : "November", "Dec" : "December"}
    
    ## entity code column index
    entityCodeColumnIndex = 10


    
    for i in range(0, len(surveyData)) :
        hoursOfOperation = []
        entityCode = surveyData.iat[i, entityCodeColumnIndex]
        for j in range(0, len(hoursOfOperationInfo)):
            seasonInfo = hoursOfOperationInfo[j]
            seasonName = seasonInfo["Season"]
            seasonStartColumnIndex = seasonInfo["SeasonStart"]
            seasonEndColumnIndex = seasonInfo["SeasonEnd"]
            startTimeColumnIndex = seasonInfo["StartTime"]
            endTimeColumnIndex = seasonInfo["EndTime"]
            closeDayColumnIndex = seasonInfo["CloseDay"]
            
            startDay=""
            startMonth = ""
            startMonthShortFrom = ""
            
            
            seasonStart = surveyData.iat[i, seasonStartColumnIndex]
            if not pd.isnull(seasonStart) and seasonStart != "" :
                try:
                    startDay,startMonthShortFrom = seasonStart.split("-")
                    
                    ## Checking month is appropriate or not
                    if type(startMonthShortFrom) is str:
                        if startMonthShortFrom in allMonths :
                            startMonth = months[startMonthShortFrom]
                        else : 
                            errorData.append("Entity Code : " + str(entityCode)+ ". Value : " + str(seasonEnd) + "(row = " + str(i) + ", column = " + str(seasonEndColumnIndex) + ")." + seasonName + " End Date should be in dd-mmm format")
                        
                    else :
                        errorData.append("Entity Code : " + str(entityCode)+ ". Value : " + str(seasonStart) + "(row = " + str(i) + ", column = " +str(seasonStartColumnIndex) + ")." +  seasonName + " Start Date should be in dd-mmm format")                    
                    
                    
                    ## Checking date is appropriate or not
                    a = int(startDay)          
                    
                except Exception:
                    errorData.append("Entity Code : " + str(entityCode)+ ". Value : " + str(seasonStart) + "(row = " + str(i) + ", column = " +str(seasonStartColumnIndex) + ")." +  seasonName + " StartDate should be in dd-mmm format")

            
            
            endDay=""
            endMonth = ""
            endMonthShortFrom = ""
            
            
            seasonEnd = surveyData.iat[i, seasonEndColumnIndex]
            if not pd.isnull(seasonEnd) and seasonEnd != "":
                try:
                    endDay,endMonthShortFrom = seasonEnd.split("-")
                    
                    ## Checking month is appropriate or not
                    if type(endMonthShortFrom) is str:
                        if endMonthShortFrom in allMonths :
                            endMonth = months[endMonthShortFrom]
                        else : 
                            errorData.append("Entity Code : " + entityCode+ ". Value : " + str(seasonEnd) + "(row = " + str(i) + ", column = " + str(seasonEndColumnIndex) + ")." + seasonName + " End Date should be in dd-mmm format")
                        
                    else :
                        errorData.append("Entity Code : " + entityCode+ ". Value : " + str(seasonEnd) + "(row = " + str(i) + ", column = " + str(seasonEndColumnIndex) + ")." + seasonName + " End Date should be in dd-mmm format")
                    
                    ## Checking date is appropriate or not
                    a = int(endDay)     
                    
                except Exception:
                    errorData.append("Entity Code : " + str(entityCode) + ". Value : " + str(seasonEnd) + "(row = " + str(i) + ", column = " + str(seasonEndColumnIndex) + ")." + seasonName + " End Date should be in dd-mmm format")
            
            startTime = surveyData.iat[i, startTimeColumnIndex]
            endTime = surveyData.iat[i, endTimeColumnIndex]
            
            ## If more than one day is closed, it will be written as comma separated way
            closeDay = surveyData.iat[i, closeDayColumnIndex]
            
            if not pd.isnull(closeDay) and closeDay != "":
                try:             
                    closeDayList = closeDay.split(",")
                    
                    for x in closeDayList :
                        x = x.strip()
                        if x not in allDays :
                            errorData.append("Entity Code : " + str(entityCode)+ ". Value : " + str(closeDay) + "(row = " + str(i) + ", column = " +str(seasonEndColumnIndex) + ")." +  seasonName + " Close Day value should have values : Sunday to Saturday. More than one value should be comma separated.")
                    
                    openDays = [item for item in allDays if item not in closeDayList]

                except Exception:
                    errorData.append("Entity Code : " + str(entityCode)+ ". Value : " + str(closeDay) + "(row = " + str(i) + ", column = " +str(seasonEndColumnIndex) + ")." +  seasonName + " Close Day value should have values : Sunday to Saturday. More than one value should be comma separated.")
            else :
                openDays = [item for item in allDays]
                
            
            ## Not create hours of Operation if all the fields are empty
            if (pd.isnull(seasonStart) or seasonStart == "") and (pd.isnull(seasonEnd) or seasonEnd == "") and (pd.isnull(startTime) or startTime == "") and (pd.isnull(endTime) or endTime == "") :
                print("No need for creating of Hours of Opeartion")
            else :
                hoursOfOperation.append(prepareHoursOfOperationJson(seasonName, startDay, startMonth, endDay, endMonth, openDays, startTime, endTime))
                
        
        allHoursOfOperation.append(json.dumps(hoursOfOperation))
        
    ## writing the data errors to file
    if isErrorSavingRequired : 
        errorfile = open('C:/Projects/MSS/Loading _AmenityData_from_Survey/Python_Codes/ClubCorp/GolfServices/error.txt', 'w')
        for item in errorData:
            errorfile.write("%s\n" % item)
    else:
        print(errorData)
    
    return allHoursOfOperation
    

In [173]:
## Getting the output File name
output_fileName = golf_config['outputFileName']

all_column_indexes = []
all_column_names = []

## getting common column information
common_column_count = len(golf_config['commonColumnInformation'])

## Getting column index from config json file
for i in range(0, common_column_count):
    all_column_indexes.append(golf_config['commonColumnInformation'][i]['columnIndex'])
    all_column_names.append(golf_config['commonColumnInformation'][i]['outputColumnName'])
    
golf_service_related_columns = []    
## getting golf service related column information
golfService_column_count = len(golf_config['golfServiceColumnInformation'])

## Getting column index from config json file
for j in range(0, golfService_column_count):
    all_column_indexes.append(golf_config['golfServiceColumnInformation'][j]['columnIndex'])
    all_column_names.append(golf_config['golfServiceColumnInformation'][j]['outputColumnName'])
    golf_service_related_columns.append(golf_config['golfServiceColumnInformation'][j]['outputColumnName'])

## Data for 1st Golf Course
golfService = surveyData.iloc[:,all_column_indexes]
golfService.columns = all_column_names
golfService['Index']=1

## getting Hours of Related column information from config file
hoursOfOperationInfo = golf_config['HoursOfOperationInformation']

hrsOfOperation = createHoursOfOperationForEachRow(hoursOfOperationInfo, surveyData, True)
golfService['HrsOfOpeartion'] = hrsOfOperation

filterCondition  = golfService['GolfServiceTakingConditionColumn'] != 'N/A'

clubsNotHavingGolfService = golfService[~filterCondition]
clubsNotHavingGolfService = clubsNotHavingGolfService.drop("Index",1)
clubsNotHavingGolfService = clubsNotHavingGolfService.drop("HrsOfOpeartion",1)

clubsNotHavingGolfService.to_csv("C:/Projects/MSS/Loading _AmenityData_from_Survey/Python_Codes/ClubCorp/GolfServices/GolfServices_ClubsNotHaving.csv")

golfService = golfService[filterCondition]
golfService['Status'] = 'Active'
golfService['Type'] = 'Golf'
golfService['Name'] = 'Golf Shop'

for x  in golf_service_related_columns:
    columnName = x
    golfService[columnName + '_Boolean'] = golfService.apply(determineBooleanValues, axis=1)

golfService.to_csv("C:/Projects/MSS/Loading _AmenityData_from_Survey/Python_Codes/ClubCorp/GolfServices/" + output_fileName)

No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for creating of Hours of Opeartion
No need for

C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
